In [313]:
import os
import pandas as pd
# from sympy import sympify

In [ ]:
N = 10
file_name = f"combinations_summary_of_{N}_panels"
extension = ".csv"
file_name_with_extension = file_name + extension
pre_path = "outputs"
full_path = os.path.join(pre_path, file_name_with_extension)
df_combinations_summary = pd.read_csv(full_path, sep=";")
df_combinations_summary

In [ ]:
one_example_configuration = df_combinations_summary["panel_configurations"].values[8]
one_example_configuration 

'A9'

In [ ]:
def symbolic_representation_to_dict (expr: str) :
    """
    Parse an expression like 'S(A1,A2,P(A3,A4))' into a nested dictionary structure.

    Examples:
        'S(A1,A2,P(A3,A4))' -> {
            'S': [
                'A1',
                'A2',
                {'P': ['A3', 'A4']}
            ]
        }
    """
    expr = expr.strip()

    # Base case: if there are no parentheses, treat this as a panel name
    if '(' not in expr and ')' not in expr:
        # Return the panel name as a simple string
        return expr

    # Otherwise, the expression must start with S(...) or P(...)
    node_type = expr[0]  # 'S' or 'P', typically
    if node_type not in ('S', 'P'):
        raise ValueError(f"Expression should start with 'S' or 'P', found '{node_type}' instead.")

    # Find the main parentheses
    start_paren = expr.index('(')
    end_paren = expr.rindex(')')
    inside = expr[start_paren + 1:end_paren].strip()

    # Split the inside by commas at the top level (ignoring nested parentheses)
    elements = []
    bracket_level = 0
    current = []

    for ch in inside:
        if ch == '(':
            bracket_level += 1
            current.append(ch)
        elif ch == ')':
            bracket_level -= 1
            current.append(ch)
        elif ch == ',' and bracket_level == 0:
            # We are at a top-level comma, so this ends one element
            element_str = ''.join(current).strip()
            if element_str:
                elements.append(element_str)
            current = []
        else:
            current.append(ch)

    # Add the last element if any
    last_element = ''.join(current).strip()
    if last_element:
        elements.append(last_element)

    # Recursively parse each sub-element
    children = [symbolic_representation_to_dict(e) for e in elements]

    # Return a dictionary: { 'S': [...]} or { 'P': [...]}
    return {node_type: children}
dictionary_representation = symbolic_representation_to_dict(one_example_configuration)
print(f"Simbolic representation: {one_example_configuration} ---> Dictionary representation: {dictionary_representation}")  


Simbolic representation: A9 ---> Dictionary representation: A9


In [ ]:
def compute_total_resistance(config, apply_shading: bool = False) -> str:
    """
    Compute the symbolic expression for the total resistance of a PV configuration.
    
    Note: Shading is assumed to have no effect on the intrinsic resistance of a panel.
    
    Parameters:
      config: A nested dictionary representing the PV configuration or a string for a panel name.
              For example: {'S': ['A1', 'A2', {'P': ['A3', 'A4']}]}
      apply_shading: (Optional) If True, shading parameters are incorporated symbolically.
                     For resistance, shading is ignored.
    
    Returns:
      A string representing the symbolic formula for total resistance.
      
    Rules:
      - Series (S): R_total = R1 + R2 + ...  
      - Parallel (P): R_total = 1 / (1/R1 + 1/R2 + ...)  
      - Panel (e.g. "A1"): returns "R_A1"
    """
    if isinstance(config, str):
        return f"R_{config}"
    
    if not isinstance(config, dict) or len(config) != 1:
        raise ValueError("Config must be a panel name (str) or a dict with one key (S/P).")
    
    node_type = next(iter(config.keys()))
    children = config[node_type]
    child_expr = [compute_total_resistance(child, apply_shading) for child in children]
    
    if node_type == 'S':
        return " + ".join(child_expr)
    elif node_type == 'P':
        inverses = [f"(1/({expr}))" for expr in child_expr]
        return f"1 / ({' + '.join(inverses)})"
    else:
        raise ValueError(f"Unknown node type '{node_type}'.")


def compute_total_voltage(config, apply_shading: bool = False) -> str:
    """
    Compute the symbolic expression for the total voltage of a PV configuration.
    
    Note: Shading is assumed not to affect the intrinsic voltage of a panel.
    
    Parameters:
      config: A nested dictionary representing the PV configuration or a string for a panel name.
      apply_shading: (Optional) If True, shading parameters are incorporated symbolically.
                     For voltage, shading is ignored.
    
    Returns:
      A string representing the symbolic formula for total voltage.
    
    Rules:
      - Series (S): V_total = V1 + V2 + ...  
      - Parallel (P): V_total = V (all branches share the same voltage, taken from the first branch)  
      - Panel (e.g. "A1"): returns "V_A1"
    """
    if isinstance(config, str):
        return f"V_{config}"
    
    if not isinstance(config, dict) or len(config) != 1:
        raise ValueError("Config must be a panel name (str) or a dict with one key (S/P).")
    
    node_type = next(iter(config.keys()))
    children = config[node_type]
    child_expr = [compute_total_voltage(child, apply_shading) for child in children]
    
    if node_type == 'S':
        return " + ".join(child_expr)
    elif node_type == 'P':
        return child_expr[0]
    else:
        raise ValueError(f"Unknown node type '{node_type}'.")


def compute_total_current(config, apply_shading: bool = False) -> str:
    """
    Compute the symbolic expression for the total current of a PV configuration.
    
    In a series branch, all panels share the same current; therefore, the expression
    of the first panel is used.
    
    When shading is applied, the effective current of an individual panel is modified
    symbolically as follows:
    
         I_effective = (G_panel / G_STC) * (1 - p_shade_panel) * I_panel
    
    where:
      - G_panel: Actual irradiance on the panel (W/m²)
      - G_STC: Standard test condition irradiance (e.g., 1000 W/m²)
      - p_shade_panel: Fraction of the panel area that is shaded (0 to 1)
    
    Parameters:
      config: A nested dictionary representing the PV configuration or a string for a panel name.
      apply_shading: (Optional) If True, the current expression for a panel includes the shading
                     model: returns "((G_{panel}/G_STC) * (1 - p_shade_{panel}) * I_{panel})".
    
    Returns:
      A string representing the symbolic formula for total current.
      
    Rules:
      - Series (S): Returns the current expression of the first child.
      - Parallel (P): I_total = I1 + I2 + ...
      - Panel (e.g. "A1"): returns "I_A1" (or the shaded version if apply_shading is True)
    """
    if isinstance(config, str):
        panel_name = config
        if apply_shading:
            return f"((G_{panel_name}/G_STC) * (1 - p_shade_{panel_name}) * I_{panel_name})"
        else:
            return f"I_{panel_name}"
    
    if not isinstance(config, dict) or len(config) != 1:
        raise ValueError("Config must be a panel name (str) or a dict with one key (S/P).")
    
    node_type = next(iter(config.keys()))
    children = config[node_type]
    child_expr = [compute_total_current(child, apply_shading) for child in children]
    
    if node_type == 'S':
        return child_expr[0]
    elif node_type == 'P':
        return " + ".join(child_expr)
    else:
        raise ValueError(f"Unknown node type '{node_type}'.")


def compute_total_power(config, apply_shading: bool = False) -> str:
    """
    Compute the symbolic expression for the total power of a PV configuration.
    
    Total power is expressed as the product of the total voltage and the effective total current.
    For an individual panel, when shading is applied, the power is expressed symbolically as:
    
         P_panel = V_panel * ((G_panel / G_STC) * (1 - p_shade_panel) * I_panel)
    
    where:
      - V_panel: Voltage of the panel.
      - I_panel: Nominal current of the panel.
      - G_panel: Actual irradiance on the panel (W/m²).
      - G_STC: Standard test condition irradiance (e.g., 1000 W/m²).
      - p_shade_panel: Fraction of the panel that is shaded (0 for no shading, 1 for full shading).
    
    Parameters:
      config: A nested dictionary representing the PV configuration or a string for a panel name.
      apply_shading: (Optional) If True, the power expression for a panel includes the shading model.
    
    Returns:
      A string representing the symbolic formula for total power.
      
    For composite configurations, the total power is computed as:
         P_total = V_total * I_total_effective
    where V_total and I_total_effective are computed recursively.
    """
    if isinstance(config, str):
        panel_name = config
        if apply_shading:
            return f"(V_{panel_name} * ((G_{panel_name}/G_STC) * (1 - p_shade_{panel_name}) * I_{panel_name}))"
        else:
            return f"(V_{panel_name} * I_{panel_name})"
    
    # For composite configurations, compute voltage and current (propagating shading) and multiply.
    total_voltage = compute_total_voltage(config, apply_shading)
    total_current = compute_total_current(config, apply_shading)
    return f"({total_voltage}) * ({total_current})"


In [ ]:
# Example prints for a given configuration
# For instance, consider the configuration corresponding to: P(A1, A2, S(A3, A4))
dictionary_representation = {'P': ['A1', 'A2', {'S': ['A3', 'A4']}]}

print("Without shading:")
print("Dictionary Representation:", dictionary_representation)
print("Total Resistance:", compute_total_resistance(dictionary_representation, apply_shading=False))
print("Total Voltage:", compute_total_voltage(dictionary_representation, apply_shading=False))
print("Total Current:", compute_total_current(dictionary_representation, apply_shading=False))
print("Total Power (V * I):", compute_total_power(dictionary_representation, apply_shading=False))

# With shading applied (apply_shading = True) -> symbolic shading factors are included.
print("\nWith shading (symbolic parameters):")
print("Dictionary Representation:", dictionary_representation)
print("Total Resistance:", compute_total_resistance(dictionary_representation, apply_shading=True))
print("Total Voltage:", compute_total_voltage(dictionary_representation, apply_shading=True))
print("Total Current:", compute_total_current(dictionary_representation, apply_shading=True))
print("Total Power (V * I):", compute_total_power(dictionary_representation, apply_shading=True))

Without shading:
Dictionary Representation: {'P': ['A1', 'A2', {'S': ['A3', 'A4']}]}
Total Resistance: 1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3 + R_A4)))
Total Voltage: V_A1
Total Current: I_A1 + I_A2 + I_A3
Total Power (V * I): (V_A1) * (I_A1 + I_A2 + I_A3)

With shading (symbolic parameters):
Dictionary Representation: {'P': ['A1', 'A2', {'S': ['A3', 'A4']}]}
Total Resistance: 1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3 + R_A4)))
Total Voltage: V_A1
Total Current: ((G_A1/G_STC) * (1 - p_shade_A1) * I_A1) + ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2) + ((G_A3/G_STC) * (1 - p_shade_A3) * I_A3)
Total Power (V * I): (V_A1) * (((G_A1/G_STC) * (1 - p_shade_A1) * I_A1) + ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2) + ((G_A3/G_STC) * (1 - p_shade_A3) * I_A3))


In [ ]:
# def evaluate_formula(formula: str, substitutions: dict):
#     """
#     Evaluate a symbolic formula using the provided variable substitutions.
    
#     Parameters:
#       formula (str): A string representing the symbolic formula.
#                      Example: "(V_A1) * ((s_A1 * I_A1) + (s_A2 * I_A2) + (s_A3 * I_A3))"
#       substitutions (dict): A dictionary where each key is a variable name (as in the formula)
#                             and the value is the value to substitute.
#                             Example: {"V_A1": 30, "s_A1": 0.9, ...}
    
#     Returns:
#       The evaluated value of the formula as a float.
#     """
#     # Convert the string formula into a Sympy expression.
#     expr = sympify(formula)
    
#     # Substitute the provided values.
#     substituted_expr = expr.subs(substitutions)
    
#     # Evaluate the expression numerically and return the result.
#     return substituted_expr.evalf()

# # Ejemplo de uso:
# formula_str = "(V_A1) * ((s_A1 * I_A1) + (s_A2 * I_A2) + (s_A3 * I_A3))"
# subs = {
#     "V_A1": 30,
#     "s_A1": 0.9,
#     "I_A1": 5,
#     "s_A2": 1,
#     "I_A2": 5,
#     "s_A3": 0.8,
#     "I_A3": 5
# }

# result = evaluate_formula(formula_str, subs)
# print("Evaluated result:", result)

In [ ]:
panel_configurations = df_combinations_summary["panel_configurations"].values
panel_configurations

array(['A1', 'A2', 'A3', ..., 'S(P(A6,A7),P(A8,A9))',
       'S(P(A6,A8),P(A7,A9))', 'S(P(A6,A9),P(A7,A8))'],
      shape=(230935,), dtype=object)

In [ ]:
symbolic_dic_representation = [symbolic_representation_to_dict(configuration) for configuration in panel_configurations]
symbolic_dic_representation

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 {'P': ['A1', 'A2']},
 {'P': ['A1', 'A2', 'A3']},
 {'P': ['A1', 'A2', 'A3', 'A4']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A9']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', {'S': ['A8', 'A9']}]},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A8']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A8', 'A9']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A8', {'S': ['A7', 'A9']}]},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A9']},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A9', {'S': ['A7', 'A8']}]},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', {'S': ['A7', 'A8']}]},
 {'P': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', {'S': ['A7', 'A8', 'A9']}]},
 {'P': ['A1', 

In [ ]:
total_resistance_formula = [compute_total_resistance(configuration) for configuration in symbolic_dic_representation]
total_resistance_formula

['R_A1',
 'R_A2',
 'R_A3',
 'R_A4',
 'R_A5',
 'R_A6',
 'R_A7',
 'R_A8',
 'R_A9',
 '1 / ((1/(R_A1)) + (1/(R_A2)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)) + (1/(R_A6)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)) + (1/(R_A6)) + (1/(R_A7)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)) + (1/(R_A6)) + (1/(R_A7)) + (1/(R_A8)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)) + (1/(R_A6)) + (1/(R_A7)) + (1/(R_A8)) + (1/(R_A9)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)) + (1/(R_A6)) + (1/(R_A7)) + (1/(R_A9)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_A4)) + (1/(R_A5)) + (1/(R_A6)) + (1/(R_A7)) + (1/(R_A8 + R_A9)))',
 '1 / ((1/(R_A1)) + (1/(R_A2)) + (1/(R_A3)) + (1/(R_

In [ ]:
total_voltage_formula = [compute_total_voltage(configuration) for configuration in symbolic_dic_representation]
total_voltage_formula

['V_A1',
 'V_A2',
 'V_A3',
 'V_A4',
 'V_A5',
 'V_A6',
 'V_A7',
 'V_A8',
 'V_A9',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 'V_A1',
 

In [ ]:
total_current_formula = [compute_total_current(configuration) for configuration in symbolic_dic_representation]
total_current_formula

['I_A1',
 'I_A2',
 'I_A3',
 'I_A4',
 'I_A5',
 'I_A6',
 'I_A7',
 'I_A8',
 'I_A9',
 'I_A1 + I_A2',
 'I_A1 + I_A2 + I_A3',
 'I_A1 + I_A2 + I_A3 + I_A4',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A8',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A8 + I_A9',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A9',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A8',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8 + I_A9',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8 + I_A7',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A9',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A9 + I_A7',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7',
 'I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8',

In [ ]:
total_power_formula = [compute_total_power(configuration) for configuration in symbolic_dic_representation]
total_power_formula

['(V_A1 * I_A1)',
 '(V_A2 * I_A2)',
 '(V_A3 * I_A3)',
 '(V_A4 * I_A4)',
 '(V_A5 * I_A5)',
 '(V_A6 * I_A6)',
 '(V_A7 * I_A7)',
 '(V_A8 * I_A8)',
 '(V_A9 * I_A9)',
 '(V_A1) * (I_A1 + I_A2)',
 '(V_A1) * (I_A1 + I_A2 + I_A3)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A8)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A8 + I_A9)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A9)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A7 + I_A8)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8 + I_A9)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A8 + I_A7)',
 '(V_A1) * (I_A1 + I_A2 + I_A3 + I_A4 + I_A5 + I_A6 + I_A9)',
 '(V_A1) * (I_A1 + I_A2 + 

In [ ]:
total_power_shading_formula = [compute_total_power(configuration,apply_shading=True) for configuration in symbolic_dic_representation]
total_power_shading_formula

['(V_A1 * ((G_A1/G_STC) * (1 - p_shade_A1) * I_A1))',
 '(V_A2 * ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2))',
 '(V_A3 * ((G_A3/G_STC) * (1 - p_shade_A3) * I_A3))',
 '(V_A4 * ((G_A4/G_STC) * (1 - p_shade_A4) * I_A4))',
 '(V_A5 * ((G_A5/G_STC) * (1 - p_shade_A5) * I_A5))',
 '(V_A6 * ((G_A6/G_STC) * (1 - p_shade_A6) * I_A6))',
 '(V_A7 * ((G_A7/G_STC) * (1 - p_shade_A7) * I_A7))',
 '(V_A8 * ((G_A8/G_STC) * (1 - p_shade_A8) * I_A8))',
 '(V_A9 * ((G_A9/G_STC) * (1 - p_shade_A9) * I_A9))',
 '(V_A1) * (((G_A1/G_STC) * (1 - p_shade_A1) * I_A1) + ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2))',
 '(V_A1) * (((G_A1/G_STC) * (1 - p_shade_A1) * I_A1) + ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2) + ((G_A3/G_STC) * (1 - p_shade_A3) * I_A3))',
 '(V_A1) * (((G_A1/G_STC) * (1 - p_shade_A1) * I_A1) + ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2) + ((G_A3/G_STC) * (1 - p_shade_A3) * I_A3) + ((G_A4/G_STC) * (1 - p_shade_A4) * I_A4))',
 '(V_A1) * (((G_A1/G_STC) * (1 - p_shade_A1) * I_A1) + ((G_A2/G_STC) * (1 - p_shade_A2) 

In [ ]:
df_combinations_summary["total_resistance_formula"] = total_resistance_formula
df_combinations_summary["total_voltage_formula"] = total_voltage_formula
df_combinations_summary["total_current_formula"] = total_current_formula
df_combinations_summary["total_power_formula"] = total_power_formula
df_combinations_summary["total_power_shading_formula"] = total_power_shading_formula

In [ ]:
df_combinations_summary.columns

Index(['panel_configurations', 'count partitions came from this configuration',
       'partitions', 'total_resistance_formula', 'total_voltage_formula',
       'total_current_formula', 'total_power_formula',
       'total_power_shading_formula'],
      dtype='object')

In [ ]:
df_combinations_summary.drop(["count partitions came from this configuration", "partitions"],axis = 1, inplace=True)
df_combinations_summary

,panel_configurations,total_resistance_formula,total_voltage_formula,total_current_formula,total_power_formula,total_power_shading_formula
0,A1,R_A1,V_A1,I_A1,(V_A1 * I_A1),(V_A1 * ((G_A1/G_STC) * (1 - p_shade_A1) * I_A1))
1,A2,R_A2,V_A2,I_A2,(V_A2 * I_A2),(V_A2 * ((G_A2/G_STC) * (1 - p_shade_A2) * I_A2))
2,A3,R_A3,V_A3,I_A3,(V_A3 * I_A3),(V_A3 * ((G_A3/G_STC) * (1 - p_shade_A3) * I_A3))
3,A4,R_A4,V_A4,I_A4,(V_A4 * I_A4),(V_A4 * ((G_A4/G_STC) * (1 - p_shade_A4) * I_A4))
4,A5,R_A5,V_A5,I_A5,(V_A5 * I_A5),(V_A5 * ((G_A5/G_STC) * (1 - p_shade_A5) * I_A5))
...,...,...,...,...,...,...
230930,"S(P(A5,A9),P(A6,A8))",1 / ((1/(R_A5)) + (1/(R_A9))) + 1 / ((1/(R_A6)...,V_A5 + V_A6,I_A5 + I_A9,(V_A5 + V_A6) * (I_A5 + I_A9),(V_A5 + V_A6) * (((G_A5/G_STC) * (1 - p_shade_...
230931,"S(P(A5,A9),P(A7,A8))",1 / ((1/(R_A5)) + (1/(R_A9))) + 1 / ((1/(R_A7)...,V_A5 + V_A7,I_A5 + I_A9,(V_A5 + V_A7) * (I_A5 + I_A9),(V_A5 + V_A7) * (((G_A5/G_STC) * (1 - p_shade_...
230932,"S(P(A6,A7),P(A8,A9))",1 / ((1/(R_A6)) + (1/(R_A7))) + 1 / ((1/(R_A8)...,V_A6 + V_A8,I_A6 + I_A7,(V_A6 + V_A8) * (I_A6 + I_A7),(V_A6 + V_A8) * (((G_A6/G_STC) * (1 - p_shade_...
230933,"S(P(A6,A8),P(A7,A9))",1 / ((1/(R_A6)) + (1/(R_A8))) + 1 / ((1/(R_A7)...,V_A6 + V_A7,I_A6 + I_A8,(V_A6 + V_A7) * (I_A6 + I_A8),(V_A6 + V_A7) * (((G_A6/G_STC) * (1 - p_shade_...


In [ ]:
# unit SI
technical_restrictions = {
    "max_idc_inv": 12, 
    "min_vdc_inv": 80,
    "max_vdc_inv": 1000, 
}
constant_values = {
    "G_STC": 1000,
}

In [ ]:
df_combinations_summary.to_csv(f"outputs/combinations_summary_of_{N}_panels_with_formulas.csv", sep=";", index=False)

In [ ]:
print(df_combinations_summary)

        panel_configurations  \
0                         A1   
1                         A2   
2                         A3   
3                         A4   
4                         A5   
...                      ...   
230930  S(P(A5,A9),P(A6,A8))   
230931  S(P(A5,A9),P(A7,A8))   
230932  S(P(A6,A7),P(A8,A9))   
230933  S(P(A6,A8),P(A7,A9))   
230934  S(P(A6,A9),P(A7,A8))   

                                 total_resistance_formula  \
0                                                    R_A1   
1                                                    R_A2   
2                                                    R_A3   
3                                                    R_A4   
4                                                    R_A5   
...                                                   ...   
230930  1 / ((1/(R_A5)) + (1/(R_A9))) + 1 / ((1/(R_A6)...   
230931  1 / ((1/(R_A5)) + (1/(R_A9))) + 1 / ((1/(R_A7)...   
230932  1 / ((1/(R_A6)) + (1/(R_A7))) + 1 / ((1/(R_A8)...   
23093